TO DO SVD:
- Try to get phrases rather than just words - perhaps beyond scope
    - Can specify bigrams in tokenizer, but then returns "australian" "australian prime" "prime" - need some cleaning
    - Also need to force both words to be nouns. else "fed says" 
- Eliminate labels with low edit distance (also hierarchial?) 
- reintroduce drop clusters with 1 document? how frequent is this?
- is there any way to visualize these clusters?? 
- once do a full run, look at statistic distributions and look at outliers 

TO DO in general:
- Visualization with tsne or umap rather than mds -- tsne works. can't get umap to install.    
- If time figure out proximity matrix and do correlation analysis

SVD generally results in more clusters because restricting more in hierarchial in where we are searching for the knee. But this is a good thing -- hierarchial can get more granular with hierarhcy, not possible with SVD   
   
Super garbage silhouette score, but not surprising: points are in multiple clusters so comparing with points also in your own cluster.    
In general, looking at how separated clusters are is not helpful for svd because can belong to multiple clusters   
    
Am currently not lemmatizing like in hiearchial. Made labels look bad. Don't know why.   
Maybe want to enforce some edit distance requirement. 

In [1]:
import nltk
from nltk.corpus import reuters
import re
import numpy as np
import pandas as pd
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
import sklearn
from scipy.cluster.hierarchy import dendrogram, linkage, ward, fcluster
import networkx as nx
import collections
import math
import operator
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.decomposition import PCA
from kneed import KneeLocator
from sklearn.manifold import TSNE
from nltk.stem.porter import PorterStemmer
import pickle
import gc

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gimli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Tf-Idf Matrix

In [2]:
def stemming_tokenizer(str_input):
    stemmer = PorterStemmer()

    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [stemmer.stem(word) for word in words]
    
    return words

In [3]:
def tf_idf(df):
    
    #stemmer = PorterStemmer()
    
    #stop_lem = [stemming_tokenizer(t) for t in stopwords.words('english')]
    #stop_lem = [item for sublist in stop_lem for item in sublist]
    
    # tfidf. stop word removal. word tokenizer. 
    #tfidf = TfidfVectorizer(stop_words = stop_lem, tokenizer = stemming_tokenizer)
    tfidf = TfidfVectorizer(stop_words = stopwords.words('english'), max_features = 5000)

    m = tfidf.fit_transform(df['text'])
    
    feature_names = tfidf.get_feature_names() # words 

    return m, feature_names

### Remove Search from TF-IDF Matrix

In [4]:
def remove_search(tfidf, feature_names, search):
    try: # sometimes search already removed (stop word)
        # remove search from the tfidf matrix: do not want as a label or clustering factor
        search_index = feature_names.index(search)
        cols = list(range(0,len(feature_names)))
        del cols[cols.index(search_index)]
        tfidf = tfidf[:,cols]
        del feature_names[search_index]
    except ValueError: 
        pass
    except:
        raise 'unknown error'
    
    return tfidf, feature_names

### SVD 

In [5]:
def svd_calculate(m):
    U, S, Vt = np.linalg.svd(m.todense(), full_matrices = False) # full_matrices make dimensions work
    V = Vt.T
    return U, S, V

### Find K

In [6]:
# variance for various values of k -> rate of change 
# preference is to choose some percent of variance, but 80% retains too many topics 
def roc_var_calculate(S):
    # calculate variance for various values of k
    k_var_lst = []
    k_var = 0
    for i in S:
        k_var += i**2
        k_var_lst.append(k_var)
            
    # rate of change of variance
    roc = []
    for k in range(len(k_var_lst)):
        if k+1 < len(k_var_lst):
            roc.append(abs(k_var_lst[k+1] - k_var_lst[k]) / k_var_lst[k])
    
    return roc

In [7]:
# Find Elbow in Variance ROC 
def find_knee(roc):
    # knee in variance ROC 
    kn = KneeLocator(range(len(roc)), roc, curve='convex', direction='decreasing')
    if kn.knee == None: # sometimes there is no knee, just take the max k in that case
        return len(roc)
    k = kn.knee + 1 # index starts at 0 -- if knee = 1, then is roc spot 2. Between k = 2 and k = 3 --> k = 2 
    #https://raghavan.usc.edu//papers/kneedle-simplex11.pdf
    
    # need at least 2 clusters
    if k < 2:
        return 2
    
    return k 

In [8]:
#fig, ax = plt.subplots()
#ax.plot(roc)
#ax.axvline(k-1, color = 'black')
#print(k)

### Reduce Dimensionality

In [9]:
def reduce_V(V, k):
    # zero out non-selected k's
    #S[k:] = 0
    V = V[:,:k] 
    # V is term to concept. Because no phrases yet, no need to multiple by a term matrix. Currently T = identity 
    
    return V 

### Find Cluster Labels

In [10]:
def find_labels(V, feature_names): # top 3 words/vectors 
    # find maximum 3 term vectors for each column (concept) in V
    # find top 3 with key = top 1, 2, 3
    max_w = dict()
    max_w_score = dict()
    for i in range(1,4):
        max_w[i] = []
        max_w_score[i] = []
        for r in range(len(V.T)):
            max_w[i].append(np.array(V.T[r])[0].argsort()[-i:][::-1][i-1])
            max_w_score[i].append(np.sort(np.array(V.T[r])[0])[-i:][::-1][i-1])
    
    # find top 3 with key = concept
    max_w3 = dict()
    max_w3_score = dict()
    max_score = dict()
    for r in range(len(V.T)):
        if r in max_w3:
            max_w3[r].append(np.array(V.T[r])[0].argsort()[-3:][::-1])
            max_w3_score[r].append((np.sort(np.array(V.T[r])[0])[-3:][::-1]))
        else:
            max_w3[r] = [np.array(V.T[r])[0].argsort()[-3:][::-1]]
            max_w3_score[r] = [np.sort(np.array(V.T[r])[0])[-3:][::-1]]
        max_score[r] = np.max(max_w3_score[r])

    # find corresponding words -> labels 
    labels = dict()
    for k,v in max_w3.items():
        labels[k] = []
        for w in v[0]:
            labels[k].append(feature_names[w])
            
    return labels, max_w, max_w3, max_w3_score, max_score
    
# label overlap only if use phrases (Z^TZ)
# TO DO: what about phrases? then could just use max 

### Find Documents in Clusters Based on Labels 
Can include any of the top 3 words that describes each cluster/concept

In [11]:
def find_docs(feature_names, max_w, m):
    # term-concept label matrix 
    # term-term matrix is identity because currently no phrases. 

    # compute for each of the top 3 words 
    # assign to cluster if any of those words exist in the document 
    Q1 = np.identity(len(feature_names))[:,max_w[1]]
    Q2 = np.identity(len(feature_names))[:,max_w[2]]
    Q3 = np.identity(len(feature_names))[:,max_w[3]]

    # cij = strength of membership of jth document to ith concept 
    C1 = np.matmul(Q1.T, m.T.toarray())
    C2 = np.matmul(Q2.T, m.T.toarray())
    C3 = np.matmul(Q3.T, m.T.toarray())
    
    # choose documents for each cluster with strength > 0 aka exists in document
    # any of the top 3 words in the cluster 
    docs = dict()
    for r in range(len(C1)):
        docs[r] = []
        for c in range(len(C1[r])): 
            if C1[r][c] > 0 or C2[r][c] > 0 or C3[r][c] > 0: # threshold 
                docs[r].append(c)

    # documents can be in multiple clusters 
    # documents can be in no clusters 

    # drop clusters with only 1 document in it 
    #del_lst = []
    #for k,v in docs.items():
    #    if len(v) == 1:
    #        del_lst.append(k)
    #for i in del_lst:
    #    del docs[i]
        
    return docs

### Combine Clusters with Overlapping Labels    
If any of the 3 label words are the same:
- Combine documents into 1 cluster
- Choose the 3 highest ranking labels between the two clusters 

Allowing multiple clusters to combine. 

In [12]:
# combine dictionaries based on combined clusters
def combine_dictionary_old(d, combo_clusters):
    del_keys = []
    for i in combo_clusters:
        d[i[0]] += d[i[1]]
        del_keys.append(i[1])
    # delete absorbed clusters
    for i in del_keys:
        del d[i]
        
    return d

In [13]:
# combine dictionaries based on combined clusters
def combine_dictionary(d, combo_clusters):
    del_keys = []
    for k,v in combo_clusters.items():
        if len(combo_clusters[k]) > 0:
            for i in v:
                d[k] += d[i]
                del_keys.append(i)
    # delete absorbed clusters
    for i in del_keys:
        del d[i]
        
    return d

In [14]:
def combine_clusters(labels, max_w3_score, max_w3,  max_score, docs, feature_names):
    
    # find clusters that contain any of the same label words
    # only allow one combination per cluster. sometimes multiple clusters with the same word
        # in the future, could do more combinations based on cleaned until convergence if deemed necessary
    combo_clusters = dict()
    used_clusters = []
    for k in labels.keys():
        combo_clusters[k] = list()
        for k2 in labels.keys():
            if k < k2 and k2 not in used_clusters: 
                intersect = set(labels[k]).intersection(set(labels[k2]))
                if len(intersect) != 0 and k2 not in used_clusters:
                    combo_clusters[k].append(k2)
                    used_clusters.append(k2)
    if len(combo_clusters) == 0:
        return docs, labels, max_w3_score, max_w3
    
    # combine scores and label indicies based on combined clusters 
    max_w3_score = combine_dictionary(max_w3_score, combo_clusters)
    max_w3 = combine_dictionary(max_w3, combo_clusters)
    max_score = combine_dictionary(max_score, combo_clusters)
    
    # flatten lists
    for k in max_w3_score.keys():
        max_w3_score[k] = [item for sublist in max_w3_score[k] for item in sublist]
    for k in max_w3.keys():
        max_w3[k] = [item for sublist in max_w3[k] for item in sublist]
        
    # generate combined labels 
    # indices in max_w3 with the max 3 scores in combined max_w3_score per cluster
    combined_labels = dict()
    count = 0
    for k in max_w3_score.keys():
        # take first 5 because then will drop repeats. limit to 3 next
        label_keys = np.array(max_w3[k])[list(np.array(max_w3_score[k]).argsort()[-5:][::-1])] 
        # find indicies in feature_arrays, unique values only
        # get indexes first to preserve order. np.unique naturally sorts. Want most important label first. 
        indexes = list(np.unique(np.array(feature_names)[label_keys], return_index = True)[1])
        combined_labels[count] = [np.array(feature_names)[label_keys][index] for index in sorted(indexes)]        
        count += 1
        
    # limit labels to first 3
    for k,v in combined_labels.items():
        combined_labels[k] = v[:3]

    # combine documents based on combined clusters 
    docs = combine_dictionary(docs, combo_clusters)
    # reset keys to logical values
    count = 0
    docs_copy = docs.copy()
    for k in docs_copy.keys():
        docs[count] = docs.pop(k)
        count += 1
        
    # reset keys to logical values
    count = 0
    max_score_copy = max_score.copy()
    for k in max_score_copy.keys():
        max_score[count] = max_score.pop(k)
        count += 1
        
    return docs, combined_labels, max_score

In [15]:
def combine_clusters_old(labels, max_w3_score, max_w3, docs, feature_names):
    
    #combo_clusters = []
    #for k in labels.keys():
    #    for k2 in labels.keys():
    #        if k > k2:
    #            intersect = set(labels[k]).intersection(set(labels[k2]))
    #            if len(intersect) != 0:
    #                combo_clusters.append([k, k2])
    
    # find clusters that contain any of the same label words
    # only allow one combination per cluster. sometimes multiple clusters with the same word
        # in the future, could do more combinations based on cleaned until convergence if deemed necessary
    combo_clusters = []
    used_clusters = []
    for k in labels.keys():
        for k2 in labels.keys():
            if k > k2 and k not in used_clusters: 
                intersect = set(labels[k]).intersection(set(labels[k2]))
                if len(intersect) != 0 and k2 not in used_clusters:
                    combo_clusters.append([k, k2])
                    used_clusters.append(k)
                    used_clusters.append(k2)
    if len(combo_clusters) == 0:
        return docs, labels, max_w3_score, max_w3
                    
    # combine scores and label indicies based on combined clusters 
    max_w3_score = combine_dictionary_old(max_w3_score, combo_clusters)
    max_w3 = combine_dictionary_old(max_w3, combo_clusters)
    
    # flatten lists
    for k in max_w3_score.keys():
        max_w3_score[k] = [item for sublist in max_w3_score[k] for item in sublist]
    for k in max_w3.keys():
        max_w3[k] = [item for sublist in max_w3[k] for item in sublist]
        
    # generate combined labels 
    # indices in max_w3 with the max 3 scores in combined max_w3_score per cluster
    combined_labels = dict()
    count = 0
    for k in max_w3_score.keys():
        # take first 5 because then will drop repeats. limit to 3 next
        label_keys = np.array(max_w3[k])[list(np.array(max_w3_score[k]).argsort()[-5:][::-1])] 
        # find indicies in feature_arrays, unique values only
        # get indexes first to preserve order. np.unique naturally sorts. Want most important label first. 
        indexes = list(np.unique(np.array(feature_names)[label_keys], return_index = True)[1])
        combined_labels[count] = [np.array(feature_names)[label_keys][index] for index in sorted(indexes)]        
        count += 1
        
    # limit labels to first 3
    for k,v in combined_labels.items():
        combined_labels[k] = v[:3]

    # combine documents based on combined clusters 
    docs = combine_dictionary_old(docs, combo_clusters)
    # reset keys to logical values
    count = 0
    docs_copy = docs.copy()
    for k in docs_copy.keys():
        docs[count] = docs.pop(k)
        count += 1
        

        
    return docs, combined_labels, max_w3_score, max_w3

# Un-Lemmatize Labels

In [16]:
def labels_unlem(df, labels):
    
    stemmer = PorterStemmer()
    
    # tfidf without lemitization
    tfidf = TfidfVectorizer(stop_words = stopwords.words('english'))
    m_norm = tfidf.fit_transform(df['text'])
    words = tfidf.get_feature_names()

    # dataframe that records words and their lemitized versions
    aux = pd.DataFrame(words, columns =['word'] )
    aux['word_stemmed'] = aux['word'].apply(lambda x : stemmer.stem(x))

    # loop through returned labels and grab the first instance of the un-lemmatized word (just any version will do)
    labels_unlem = dict()
    for i in labels.keys():
        labels_unlem[i] = []
        for j in labels[i]:
            labels_unlem[i].append(aux[aux.word_stemmed == j].word.values[0])
            
    return labels_unlem

## Assign Clusters in DF 

In [17]:
def cluster_df(df, docs):
    # create dataframe that indicates which documents belong to which cluster and labels. List of clusters. 
    framesvd = df
    #framesvd['label'] = ''
    framesvd['cluster'] = ''
    for k,v in docs.items():
        for d in v: 
            framesvd.cluster = np.where(framesvd.index == d, framesvd.cluster + str(k) + ',', framesvd.cluster)
            #framesvd.label = np.where(framesvd.index == d, str(labels[k]), framesvd.label)
                    # this only gets one of th clusters in the list. not all of them. 

    # create list
    framesvd.cluster = framesvd.cluster.str.split(',')
    # drop blanks
    framesvd.cluster = framesvd.cluster.apply(lambda row: [i for i in row if i != ''])
    # drop duplicates
    framesvd.cluster = framesvd.cluster.apply(set)
    framesvd.cluster = framesvd.cluster.apply(list)
    # integers
    framesvd.cluster = framesvd.cluster.apply(lambda row: [int(i) for i in row])
    
    return framesvd

In [18]:
def dist_calculate(m):
    dist = euclidean_distances(m)  
    return dist

## Sort Clusters

In [19]:
def label_sort(labels, max_score):
    max_score = sorted(max_score.items(), key=operator.itemgetter(1), reverse = True)
    
    labels_sorted = dict()
    for i in max_score:
        labels_sorted[i[0]] = labels[i[0]]
        
    return labels_sorted

# Calculate Metrics

__Find Cluster Centroids__

In [20]:
def find_centroids(frameh, m, k):
    # most common words in clusters (based on tf-idf not just frequency)
    centroid = dict()
    for c in range(k):
        ## centroid ## 
        cluster1 = list(frameh[frameh.cluster.map(set([c]).issubset)].index.unique())
        # find documents cluster
        m1 = m[cluster1,:]
        # take mean vector among all documents
        m1 = m1.mean(axis = 0)
        # record mean vector: centroids of each sub cluster
        centroid[c] = m1
        
    return centroid

__Calculate Silhouette__

In [ ]:
def silhouette_individ(frameh, dist, k):
    # average distance to points in your cluster
    sil_a = dict()
    for c in range(k):
        sil_a[c] = dict()
        docs_i = list(frameh[frameh.cluster.map(set([c]).issubset)].index.unique())
        if len(docs_i) == 1:
            sil_a[c][docs_i[0]] = 0
        else:
            for i in docs_i.copy():
                docs_i.remove(i)
                sil_a[c][i] = np.nanmean(dist[i,docs_i].tolist())

    # minimum average distance to points in other clusters 
    sil_b = dict()
    for c in range(k):
        sil_b[c] = dict()
        docs_in = list(frameh[frameh.cluster.map(set([c]).issubset)].index.unique())
        for i in docs_in:
            # loop through other clusters and find average distance 
            lst = []
            for c2 in range(k):
                if c2 != c:
                    docs_out = list(frameh[frameh.cluster.map(set([c2]).issubset)].index.unique())
                    if i in docs_out: # can be in multiple clusters
                        docs_out.remove(i)
                    lst.append(np.nanmean(dist[i,docs_out].tolist()))
                
            # take minimum of average distance to other clusters
            sil_b[c][i] = np.min(lst)
            
    return sil_a, sil_b

In [ ]:
def silhouette_avg(frameh, dist, k):
    sil_a, sil_b = silhouette_individ(frameh, dist, k)
    
    # find silhouette score of each point in each cluster and take average -> cluster score
    sil_scores = dict()
    for k,v in sil_a.items():
        lst = []
        for i in range(len(v.values())):
            max_ab = max(list(sil_b[k].values())[i], list(sil_a[k].values())[i])
            min_ab = min(list(sil_b[k].values())[i], list(sil_a[k].values())[i])
            lst.append(1 - min_ab/max_ab)
        sil_scores[k] = np.nanmean(lst) # ignore nans: ex point in all of the clusters, so no b to calculate 
    # return overall average
    return np.nanmean(list(sil_scores.values()))

__Calculate Distortion__

In [ ]:
def distortion_calculate(m, centroid, frameh):
    sumd = 0
    countpts = 0
    for i in list(frameh.index.unique()):
        for c in frameh[frameh.index == i].cluster.tolist()[0]:
            sumd += np.linalg.norm(m[i]-centroid[c])
            countpts +=1 
        
    return sumd, sumd/countpts

# Main Function

In [ ]:
def main():
    
    # read in data
    df = pd.read_pickle('reuters_processed')
    
    # set of topics
    topics = list(df.categories)
    topics = [item for sublist in topics for item in sublist]
    topics = list(set(topics))

    percent_zero_dict = dict()
    df_final = pd.DataFrame()
    labels_dict = dict()
    k_dict = dict()
    silhouette_dict = dict()
    distortion_dict = dict()
    max_score_dict = dict()
    cluster1 = []

    for search in topics:
        gc.collect()
        
        df_subset = df[df.categories.map(set([search]).issubset)] 
        df_subset = df_subset.reset_index()
        
        if len(df_subset) < 5:
            cluster1.append(search)
            continue
            
        if search == 'earn' or search == 'acq': # too big to deal with
            continue
        
        print(search)
        
        # TF-IDF matrix
        tfidf, feature_names = tf_idf(df_subset)
                
        # remove search from tf-idf matrix
        tfidf, feature_names = remove_search(tfidf, feature_names, search)

        # SVD 
        U, S, V = svd_calculate(tfidf)
        # Find K and reduce dimensionality
        roc = roc_var_calculate(S)
        k = find_knee(roc) ## alternative is to find_knee(S) directly: size of singular values drops off
                            ## but this often results in even more clusters. not a good thing.
        V = reduce_V(V, k)

        # Find cluster labels
        labels, max_w, max_w3, max_w3_score, max_score = find_labels(V, feature_names)
        #return labels
        # Assign documents to clusters based on labels
        docs = find_docs(feature_names, max_w, tfidf)
        # combine clusters with overlapping labels
        docs, labels, max_score = combine_clusters(labels, max_w3_score, max_w3, max_score, docs, feature_names)

        # unlemmatize labels
        #labels = labels_unlem(df_subset, labels)
        
        # sort clusters
        labels = label_sort(labels, max_score)
        
        # mark clusters in dataframe
        frame = cluster_df(df_subset, docs)
        # percent of documents with no cluster
        frame['len']= frame.cluster.apply(lambda row: len(row))
        percent_zero_dict[search] = len(frame[frame.len == 0]) / len(frame)

        if len(labels) == 1:
            cluster1.append(search)
            continue
        
        # calculate metrics
        dist = dist_calculate(tfidf)
        centroid = find_centroids(frame, tfidf, len(labels))
        silhouette = silhouette_avg(frame, dist, len(labels))
        distortion, distortion_avg = distortion_calculate(tfidf, centroid, frame)
        
        labels_dict[search] = labels
        k_dict[search] = len(labels) # don't use k because combined clusters after that step
        silhouette_dict[search] = silhouette
        distortion_dict[search] = distortion_avg
        max_score_dict[search] = max_score

        frame['search'] = search
        df_final = df_final.append(frame)
        
    return df_final, labels_dict, percent_zero_dict, k_dict, distortion_dict, silhouette_dict, cluster1

In [ ]:
df_final, labels_dict, percent_zero_dict, k_dict, distortion_dict, silhouette_dict, cluster1 = main()

In [ ]:
with open('lingo', "wb") as f:
    pickle.dump(df_final, f)
    pickle.dump(labels_dict, f)
    pickle.dump(k_dict, f)
    pickle.dump(distortion_dict, f)
    pickle.dump(silhouette_dict, f)
    pickle.dump(percent_zero_dict, f)

In [ ]:
k_dict

In [ ]:
pd.DataFrame(list(percent_zero_dict.values())).describe()

In [ ]:
pd.DataFrame(list(distortion_dict.values())).describe()

In [ ]:
pd.DataFrame(list(silhouette_dict.values())).describe()

In [ ]:
silhouette_dict